In [1]:
# Import dependencies
# Standard python libraries
import json
import os
# Third-party libraries
from flask import Flask, redirect, request, url_for, render_template, session
from flask_login import (
    UserMixin,
    LoginManager,
    current_user,
    login_required,
    login_user,
    logout_user,
)
from bs4 import BeautifulSoup, ProcessingInstruction
from oauthlib.oauth2 import WebApplicationClient
import requests
import pandas as pd
import psycopg2
import plotly
import plotly.express as px
import plotly.graph_objects as go

# Internal imports
from user import User
from db import get_df
from mfl import get_mfl, get_mfl_liveScoring, get_mfl_league

# Configuration (These variables are stored as environment variables)
GOOGLE_CLIENT_ID = os.environ.get("GOOGLE_CLIENT_ID", None)
GOOGLE_CLIENT_SECRET = os.environ.get("GOOGLE_CLIENT_SECRET", None)
GOOGLE_DISCOVERY_URL = ("https://accounts.google.com/.well-known/openid-configuration")

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

In [2]:
user_league = "53906"

In [29]:
# Get MFL scoring data
liveScores = get_mfl_liveScoring(user_league)
liveScores

,franchiseID,id_mfl,liveScore,secondsRemaining,status
0,0009,13671,6.5,0,starter
1,0009,11945,1.0,0,starter
2,0009,15241,6.7,0,starter
3,0009,13129,0.0,3600,starter
4,0009,11228,1.6,0,starter
...,...,...,...,...,...
175,0006,14558,1.9,0,starter
176,0006,15329,2.4,0,starter
177,0006,13590,15.7,0,starter
178,0006,12626,10.3,0,starter


In [30]:

# Get Franchises in the league
franchises = get_mfl_league(user_league)
franchises

,franchiseID,franchiseName
0,0001,Providence Monastic Spirits
1,0002,Washington Fantasy Football Team
2,0003,Verdansk Vaqueros FFC
3,0004,Down with the Sickness unto Death
4,0005,Crocodilopolis Body Snatchers
5,0006,O'Henry's Stiffies
6,0007,Comeback Kings
7,0008,WHAM! BAM! Thank You CAM!
8,0009,Pretty Big Wieners
9,0010,Saskatoon Squatches


In [31]:
# Get all players, sharkRank, and ADP
predictions = get_df("predictions")
predictions


,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,78.0,60.66,347.798672,145.069999,302.669999,136.640001,294.240001
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,63.0,29.60,341.323461,178.046668,335.646667,178.046668,335.646667
2,13116,PATRICK MAHOMES,26.0,KCC,QB,QB1,NO,NO,NO,55.0,26.56,339.347453,193.409999,351.009999,193.409999,351.009999
3,12620,DAK PRESCOTT,29.0,DAL,QB,QB1,NO,NO,NO,75.0,71.23,332.541423,153.929998,311.529998,118.080001,275.680001
4,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,118.0,84.10,331.415001,67.943333,225.543333,102.406669,260.006669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,13308,JAMAL AGNEW,27.0,JAC,WR,WR2,KR1,PR1,NO,747.0,3000.00,109.342865,-47.473333,152.126667,-85.246667,114.353333
615,14838,LAVISKA SHENAULT,23.0,CAR,WR,WR3,NO,NO,NO,827.0,3000.00,24.121850,-51.323333,64.276667,-83.530000,32.070000
616,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR2,KR1,PR1,NO,1284.0,3000.00,107.820542,-86.543333,113.056667,-54.880000,144.720000
617,12394,DEANDRE CARTER,29.0,LAC,WR,WR2,KR1,PR1,NO,1044.0,3000.00,105.890985,-59.080001,140.519999,-94.883334,104.716666


In [32]:
# merge predictions, franchises, and liveScores
merged = liveScores.merge(franchises, how='left', on='franchiseID').merge(predictions, how='left', on='id_mfl')

In [33]:
merged['liveScore'] = merged.liveScore.astype('float64')
merged['secondsRemaining'] = merged.secondsRemaining.astype('float64')

In [34]:
# calculate scoreRemaining
merged['weeklyPred'] = merged['pred'] / 17

def calcScoreRemaining(row):
    result = ((row['weeklyPred']) * (row['secondsRemaining'] / 3600)) + row['liveScore']
    return result

merged['scoreRemaining'] = merged.apply(calcScoreRemaining, axis=1)
merged


,franchiseID,id_mfl,liveScore,secondsRemaining,status,franchiseName,player,age,team,pos,...,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute,weeklyPred,scoreRemaining
0,0009,13671,6.5,0.0,starter,Pretty Big Wieners,MARK ANDREWS,26.0,BAL,TE,...,NO,33.0,25.89,196.051249,127.150002,186.850002,127.150002,186.850002,11.532426,6.50000
1,0009,11945,1.0,0.0,starter,Pretty Big Wieners,CAIRO SANTOS,30.0,CHI,PK,...,NO,560.0,172.77,136.911686,37.810001,138.810001,18.643334,119.643334,8.053629,1.00000
2,0009,15241,6.7,0.0,starter,Pretty Big Wieners,MAC JONES,24.0,NEP,QB,...,NO,234.0,138.18,259.548074,37.893331,195.493331,22.279997,179.879997,15.267534,6.70000
3,0009,13129,0.0,3600.0,starter,Pretty Big Wieners,LEONARD FOURNETTE,27.0,TBB,RB,...,NO,7.0,22.11,230.187649,131.549997,250.849997,93.793333,213.093333,13.540450,13.54045
4,0009,11228,1.6,0.0,starter,Pretty Big Wieners,ROBERT WOODS,30.0,TEN,WR,...,NO,94.0,97.63,181.939059,24.013332,139.613332,22.739999,138.339999,10.702298,1.60000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0006,14558,1.9,0.0,starter,O'Henry's Stiffies,DEONTE HARTY,24.0,NOS,WR,...,NO,356.0,3000.00,122.209997,-23.856666,175.743334,-91.863333,107.736667,7.188823,1.90000
176,0006,15329,2.4,0.0,starter,O'Henry's Stiffies,KYLE PITTS,21.0,ATL,TE,...,NO,109.0,38.28,157.924377,67.749999,127.450000,103.826669,163.526669,9.289669,2.40000
177,0006,13590,15.7,0.0,starter,O'Henry's Stiffies,BAKER MAYFIELD,27.0,CAR,QB,...,NO,768.0,143.75,254.643395,29.506664,187.106664,16.359999,173.959999,14.979023,15.70000
178,0006,12626,10.3,0.0,starter,O'Henry's Stiffies,DERRICK HENRY,28.0,TEN,RB,...,NO,5.0,4.97,279.270756,146.439995,265.739995,163.976664,283.276664,16.427692,10.30000


In [35]:
merged['diff'] = merged.scoreRemaining - merged.weeklyPred
merged.loc[merged['diff']>30, 'diff'] = 30
merged.loc[merged['diff']<-30, 'diff'] = -30

merged['scaled'] = round(merged['diff'] * 255 / 30, 0)
merged.dropna(inplace=True)
merged['scaled'] = merged['scaled'].astype('int')

In [36]:
def colorPicker(row):
    scalar = row['scaled']
    if scalar >= 0:
        red = 255 - scalar
        green = 255
        blue = 255 - scalar
    else:
        red = 255 
        green = 255 + scalar
        blue = 255 + scalar
    color = f'rgb({red},{green},{blue})'
    return color
merged['color'] = merged.apply(colorPicker, axis=1)
merged

,franchiseID,id_mfl,liveScore,secondsRemaining,status,franchiseName,player,age,team,pos,...,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute,weeklyPred,scoreRemaining,diff,scaled,color
0,0009,13671,6.5,0.0,starter,Pretty Big Wieners,MARK ANDREWS,26.0,BAL,TE,...,196.051249,127.150002,186.850002,127.150002,186.850002,11.532426,6.50000,-5.032426,-43,"rgb(255,212,212)"
1,0009,11945,1.0,0.0,starter,Pretty Big Wieners,CAIRO SANTOS,30.0,CHI,PK,...,136.911686,37.810001,138.810001,18.643334,119.643334,8.053629,1.00000,-7.053629,-60,"rgb(255,195,195)"
2,0009,15241,6.7,0.0,starter,Pretty Big Wieners,MAC JONES,24.0,NEP,QB,...,259.548074,37.893331,195.493331,22.279997,179.879997,15.267534,6.70000,-8.567534,-73,"rgb(255,182,182)"
3,0009,13129,0.0,3600.0,starter,Pretty Big Wieners,LEONARD FOURNETTE,27.0,TBB,RB,...,230.187649,131.549997,250.849997,93.793333,213.093333,13.540450,13.54045,0.000000,0,"rgb(255,255,255)"
4,0009,11228,1.6,0.0,starter,Pretty Big Wieners,ROBERT WOODS,30.0,TEN,WR,...,181.939059,24.013332,139.613332,22.739999,138.339999,10.702298,1.60000,-9.102298,-77,"rgb(255,178,178)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0006,14558,1.9,0.0,starter,O'Henry's Stiffies,DEONTE HARTY,24.0,NOS,WR,...,122.209997,-23.856666,175.743334,-91.863333,107.736667,7.188823,1.90000,-5.288823,-45,"rgb(255,210,210)"
176,0006,15329,2.4,0.0,starter,O'Henry's Stiffies,KYLE PITTS,21.0,ATL,TE,...,157.924377,67.749999,127.450000,103.826669,163.526669,9.289669,2.40000,-6.889669,-59,"rgb(255,196,196)"
177,0006,13590,15.7,0.0,starter,O'Henry's Stiffies,BAKER MAYFIELD,27.0,CAR,QB,...,254.643395,29.506664,187.106664,16.359999,173.959999,14.979023,15.70000,0.720977,6,"rgb(249,255,249)"
178,0006,12626,10.3,0.0,starter,O'Henry's Stiffies,DERRICK HENRY,28.0,TEN,RB,...,279.270756,146.439995,265.739995,163.976664,283.276664,16.427692,10.30000,-6.127692,-52,"rgb(255,203,203)"


In [37]:
# chart
players_onthefield = merged.loc[merged.status=="starter"]
players_onthefield = players_onthefield.sort_values(by='scoreRemaining', ascending=False, ignore_index=True)

fran_rank = players_onthefield.groupby('franchiseName').sum().sort_values(by='scoreRemaining', ascending=False)

sorter = fran_rank.index

players_onthefield.franchiseName = players_onthefield.franchiseName.astype("category")
players_onthefield.franchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["franchiseName"], inplace=True)
color_discrete_map = dict(zip(players_onthefield.id_mfl, players_onthefield.color))
# Create bar chart
figPred = px.bar(players_onthefield, 
            x="franchiseName", 
            y="scoreRemaining", 
            color="player", 
            color_discrete_sequence=list(players_onthefield['color']),
            #     "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
            #     "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
            #     "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
            #     "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
            #     "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
            #     "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            text='player', 
            hover_name="player",
            hover_data={
                'scoreRemaining':True, 'weeklyPred':True, 'scaled':True,
                'player':False, 'pos':False, 'franchiseName':False
                },
            labels={
                "franchiseName":"Franchise",
                "predRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figPred.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="ChopBlock Predictions",
            font_family="Skia",
            showlegend=False
            )
figPred.show()
#graphJSON_pred = json.dumps(figPred, cls=plotly.utils.PlotlyJSONEncoder)
#return render_template('compareFranchises2.html', graphJSON_pred=graphJSON_pred, graphJSON_adp=graphJSON_adp, graphJSON_shark=graphJSON_shark)


/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2631: FutureWarning:

The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.

